https://www.kaggle.com/code/prashant268/sentiment-analysis-lstm \\
https://www.kaggle.com/code/caiyutiansg/twitter-sentiment-analysis-with-word2vec-lstm

In [ ]:
# Imports
import csv
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gensim as gs
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import torch

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/Shareddrives/'Curriculum Project'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/Shareddrives/Curriculum Project


In [ ]:
# use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Parsing Data:

In [ ]:
# Use processed data
df = pd.read_csv("processed/processedtweets.csv", encoding="latin-1", header=None)

In [ ]:
# Add headers
headers = ['target', 'id', 'date', 'flag', 'username', 'tweet']
df.columns = headers

In [ ]:
# Drop unused columns
df = df.drop('id', axis=1).drop('date', axis=1).drop('flag', axis=1).drop('username', axis=1)
df = df[df['tweet'].notnull()]

Splitting into Testing and Training Data

In [ ]:
traindf, testdf = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
traindf.head()

,target,tweet
382524,0,sad sun run away
1574589,4,finished filming quest stay tuned find im talking
910361,4,stocked tonight
423343,0,needs bestie wheres pep talk
1330471,4,spend night big brother winner rachel Â£500


In [ ]:
# User Word2Vec to encode words

# Split into list of lists
tweetLists = []
for tweet in traindf.tweet:
  tweetLists.append(tweet.split())


Training the Word2Vec Model:

In [ ]:
# Use gensim models
# https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92
w2v = gs.models.word2vec.Word2Vec(size=256, window=8, min_count=8, workers=12, sg=1)
w2v.build_vocab(tweetLists)

In [ ]:
# Train model
# Took 1 hour 1 min to train with 50 epochs!
numEpochs=50
w2v.train(tweetLists, total_examples=len(tweetLists), epochs=numEpochs)

(418933568, 496000550)

In [ ]:
# Save model
w2v.save("w2v_model")